In [ ]:
# !pip3 install prompt_toolkit
!pip3 install requests
!pip3 install python-dotenv
# !pip install gspread
# !pip install oauth2client
# !pip install google-auth
# !pip install google-auth-oauthlib
# !pip install google-auth-httplib2

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import os
import csv
import requests

In [ ]:
load_dotenv('/content/.env')

In [ ]:
# Replace with your Canvas API token
access_token = os.getenv('CANVAS_ACCESS_TOKEN')
base_url = os.getenv('CANVAS_BASE_UR')

# Headers for authentication
headers = {
    'Authorization': f'Bearer {access_token}'
}

In [ ]:
# Get all conversations
conversations_response = requests.get(f'{base_url}/conversations', headers=headers)
conversations = conversations_response.json()

In [ ]:
# List to store all messages with sender names
messages_list = []

# Get messages for each conversation and extract sender names and message bodies
for conversation in conversations:
    conversation_id = conversation['id']
    messages_response = requests.get(f'{base_url}/conversations/{conversation_id}/messages', headers=headers)
    messages = messages_response.json()
    for message in messages:
        sender_name = message['author']['name']
        body = message['body']
        messages_list.append([sender_name, body])

In [ ]:
# Write to CSV file
with open('canvas_messages.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Sender', 'Message Body'])
    writer.writerows(messages_list)

print("Messages have been written to canvas_messages.csv")

In [ ]:
# Google Sheets API setup
scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

In [ ]:
# Authenticate and create the service
creds = None
if 'credentials.json' in os.listdir('/content/drive/My Drive/'):
    creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/My Drive/credentials.json', scope)
else:
    creds, _ = default()

In [ ]:
client = gspread.authorize(creds)

# Upload CSV to Google Drive and convert to Google Sheet
spreadsheet = client.create('Canvas Messages')
worksheet = spreadsheet.sheet1
with open(csv_file_path, 'r', encoding='utf-8') as file:
    content = file.read()
client.import_csv(spreadsheet.id, content)

print(f"CSV file has been uploaded and converted to a Google Sheet: {spreadsheet.url}")